In [ ]:
!pip install transformers accelerate bitsandbytes

In [ ]:
!pip install optimum

In [ ]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

In [ ]:
import pandas as pd
import io
from google.colab import files
import optimum

from transformers import AutoModelForCausalLM, AutoTokenizer
from ast import Str

from tqdm.notebook import tqdm
tqdm.pandas()

from google.colab import userdata
import os

access_token = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

### Load csv files
- Bad prompts contains no context and negative context
- Good prompts contains positive context
- Files can be found here:
https://drive.google.com/drive/folders/17wfyv9uhyxjTbbcTXKscjsXAKfDiHkoO

In [ ]:
df_gender_bad_prompts = pd.read_csv(".../gender_bad_prompts.csv", sep = ";")
df_gender_bad_prompts

In [ ]:
df_gender_good_prompts = pd.read_csv(".../gender_good_prompts.csv", sep = ";")
df_gender_good_prompts

## Load LLM: Wizard-Vicuna-7B-Uncensored-GPTQ

In [ ]:
model_wizard_name = "TheBloke/Wizard-Vicuna-7B-Uncensored-GPTQ"

model_wizard = AutoModelForCausalLM.from_pretrained(model_wizard_name, device_map="auto", use_auth_token=access_token)
tokenizer_wizard = AutoTokenizer.from_pretrained(model_wizard_name, use_auth_token=access_token, device_map="auto")

In [ ]:
def get_answer_wizard(row) -> Str:
    if type(row["context"]) == float:
      prompt = row["question"]
    else:
      prompt = row["context"] + " " + row["question"]

    model_inputs = tokenizer_wizard(prompt, return_tensors="pt")

    model_inputs = {key: tensor.to(model_wizard.device) for key, tensor in model_inputs.items()}

    output = model_wizard.generate(**model_inputs,
                            do_sample=True,
                            pad_token_id=tokenizer_wizard.eos_token_id,
                            min_new_tokens=50,
                            max_new_tokens=200,
                            temperature=0.7,
                            top_k=50,
                            top_p=0.95,
                            num_return_sequences=1)

    answer = tokenizer_wizard.decode(output[0], skip_special_tokens=True)
    return answer

Used function to generate answers in parts as it took too long otw.

In [ ]:
start = 0
end = 5000
diff = end - start
prompt_df = df_gender_bad_prompts.iloc[start:end,:].reset_index(drop=True)
step = 100
for index in tqdm(range(0, diff, step)):
  prompt_df.loc[index:index+step-1,"answer"] = prompt_df.iloc[index:index+step,:].progress_apply(get_answer_wizard, axis = 1)
  prompt_df.to_csv(".../Answers_bad_prompts/llm_wizard_responses_{}_{}.csv".format(start,end), index = False, sep = ";")

Answers are stored here: https://drive.google.com/drive/folders/1GK8_921YhJmuGLQOBT_82nklEen22YV0?usp=drive_link